In [222]:
import numpy as np
import pyfastchem 

import pyfastchem
import numpy as np
import os
import matplotlib.pyplot as plt
from astropy import constants as const


#Do the chemistry calculations
#this loads the temperatures and pressures produced by petitRADTRANS, you may need to modify these lines if you store these data products somewhere else
temperatures = np.load('data_products/radtrans_temperature.npy')
pressures = np.load('data_products/radtrans_pressure.npy')

In [223]:
fastchem = pyfastchem.FastChem('/home/calder/Documents/FastChem-master/input/element_abundances_solar.dat', '/home/calder/Documents//FastChem-master/input/logK.dat', 1)


FastChem initialisation summary:
  number of species: 523    elements: 28    molecules: 495    chemical elements: 86



In [224]:
input_data = pyfastchem.FastChemInput()
output_data = pyfastchem.FastChemOutput()

In [225]:
input_data.temperature = temperatures
input_data.pressure = pressures

In [226]:
fastchem_flag = fastchem.calcDensities(input_data, output_data)
print("FastChem reports:", pyfastchem.FASTCHEM_MSG[fastchem_flag])

FastChem reports: convergence ok


In [227]:
number_densities = np.array(output_data.number_densities)
gas_number_density = pressures*1e6 / (const.k_B.cgs * temperatures)

In [228]:
number_densities

array([[4.27563736e+07, 7.14703142e-02, 3.85370768e+04, ...,
        2.07864556e-17, 5.56459328e+02, 2.84272458e-14],
       [4.82938900e+07, 1.01893779e-01, 4.86418334e+04, ...,
        3.34730155e-17, 7.02260761e+02, 4.57707719e-14],
       [5.45676312e+07, 1.45314726e-01, 6.13943086e+04, ...,
        5.39387765e-17, 8.86222109e+02, 7.37438134e-14],
       ...,
       [1.75450806e+11, 5.02978704e+13, 1.01744279e+15, ...,
        8.49281272e+03, 2.00943565e-02, 1.56385707e+04],
       [1.94903046e+11, 5.40920962e+13, 1.28355431e+15, ...,
        1.11539522e+04, 2.32607118e-02, 2.18988222e+04],
       [2.16398674e+11, 5.81533418e+13, 1.61913909e+15, ...,
        1.46075476e+04, 2.70720388e-02, 3.06403308e+04]])

In [229]:
#change the below lines to select which species you want to calculate the volume mixing ratios for
# abundance_species_labels = ['CO', 'H2O', 'VO', 'OH', 'CO2', 'CH4', 'C2H2', 'NH3', 'HCN', 'Fe', 'Fe+', 'Cr', 'Si']
# abundance_species = ['C1O1', 'H2O1', 'O1V1', 'H1O1', 'C1O2', 'C1H4', 'C2H2', 'H3N1', 'C1H1N1_1', 'Fe', 'Fe1+', 'Cr', 'Si']

In [230]:
abundance_species_labels = ['TiO']
abundance_species = ['TiO']

In [231]:
#set the quench pressure to 1 bar
quench = np.argmin(np.abs(pressures-1e1))

In [232]:
a_index = []
abundance_species_indices, abundance_species_masses_ordered = [], []
n_species = fastchem.getSpeciesNumber()

In [233]:
for i, species in enumerate(abundance_species):
        index = fastchem.getSpeciesIndex(species)
if index != pyfastchem.FASTCHEM_UNKNOWN_SPECIES:
    abundance_species_indices.append(index)
    abundance_species_masses_ordered.append(fastchem.getSpeciesMolecularWeight(index))
    this_species = number_densities[quench, index]/gas_number_density[quench]
    print("The VMR for ",species,' is ', this_species)
else:
    print("Species", species, "to plot not found in FastChem")

Species TiO to plot not found in FastChem


In [234]:
plt.plot(number_densities[:, index]/gas_number_density[:],pressures)
plt.xscale('log')
plt.yscale('log')

IndexError: index 9999999 is out of bounds for axis 1 with size 523